In [ ]:
import pandas as pd
import numpy as np
import sys
import os
import seaborn as sns
import matplotlib.pyplot as plt

# Configuración (puedes ajustar estos parámetros)
CSV_PATHS = ['data/demographic.csv', '../data/demographic.csv', '../../data/demographic.csv']
TARGET_COL = 'INDFMPIR'  # Columna objetivo para correlaciones
MIN_ABS_CORR_IMPORTANT = 0.3  # Umbral de correlación para considerar 'importante'
RANDOM_STATE = 42

# 1. Localizar el archivo CSV
csv_file = None
for p in CSV_PATHS:
    if os.path.exists(p):
        csv_file = p
        break
if csv_file is None:
    print(f"Error: No se encontró demographic.csv en rutas: {CSV_PATHS}")
    sys.exit(1)
print(f"Usando archivo: {csv_file}")

# 2. Leer CSV
print("Leyendo el CSV...")
df = pd.read_csv(csv_file)
original_columns = df.columns.tolist()

# 3. Normalizar missing values (empty strings -> NaN)
df.replace(['', ' '], pd.NA, inplace=True)

# 4. Asegurar que la columna objetivo existe
if TARGET_COL not in df.columns:
    print(f"Advertencia: La columna objetivo '{TARGET_COL}' no está en el dataset. Solo se aplicarán reglas por % de missing.")

# 5. Calcular % de missing por columna
missing_counts = df.isna().sum()
missing_pct = missing_counts / len(df)

# 6. Preparar correlaciones respecto a TARGET_COL (solo numéricas)
if TARGET_COL in df.columns:
    # Convertir a numérico lo que se pueda
    numeric_df = df.select_dtypes(include=[np.number]).copy()
    # Intento de conversión suave para columnas object
    for col in df.columns:
        if col not in numeric_df.columns:
            try:
                numeric_df[col] = pd.to_numeric(df[col])
            except Exception:
                pass
    if TARGET_COL in numeric_df.columns:
        corr_series = numeric_df.corr(numeric_only=True)[TARGET_COL].drop(labels=[TARGET_COL], errors='ignore')
    else:
        corr_series = pd.Series(dtype=float)
        print(f"No se puede calcular correlaciones: '{TARGET_COL}' no es numérica o convertible.")
else:
    corr_series = pd.Series(dtype=float)

# 7. Visualización de la matriz de correlaciones (solo numéricas) si procede
if len(df.select_dtypes(include=[np.number]).columns) > 1:
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.select_dtypes(include=[np.number]).corr(numeric_only=True), cmap='coolwarm', center=0, annot=False)
    plt.title('Matriz de correlaciones (numéricas)')
    plt.tight_layout()
    plt.show()
else:
    print("No hay suficientes columnas numéricas para un heatmap de correlaciones.")

# 8. Decisión de eliminación según reglas
cols_drop = []
reasons = {}

for col in df.columns:
    if col == TARGET_COL:
        continue
    pct = missing_pct[col]
    corr_val = corr_series.get(col, np.nan)
    abs_corr = abs(corr_val) if not np.isnan(corr_val) else np.nan

    # Reglas:
    # > 50% eliminar siempre
    if pct > 0.50:
        cols_drop.append(col)
        reasons[col] = f"Missing {pct:.1%} > 50%"
        continue
    # > 40%-50% eliminar salvo justificación fuerte -> aquí eliminamos
    if 0.40 < pct <= 0.50:
        cols_drop.append(col)
        reasons[col] = f"Missing {pct:.1%} entre 40%-50% (regla general eliminar)"
        continue
    # 20%-40% mantener solo si muy relevante (|corr| >= MIN_ABS_CORR_IMPORTANT)
    if 0.20 < pct <= 0.40:
        if np.isnan(abs_corr) or abs_corr < MIN_ABS_CORR_IMPORTANT:
            cols_drop.append(col)
            reasons[col] = f"Missing {pct:.1%} y |corr| {abs_corr if not np.isnan(abs_corr) else 'NaN'} < {MIN_ABS_CORR_IMPORTANT}"
        continue
    # 5%-20% eliminar si NO está correlacionada (|corr| < umbral)
    if 0.05 < pct <= 0.20:
        if np.isnan(abs_corr) or abs_corr < MIN_ABS_CORR_IMPORTANT:
            cols_drop.append(col)
            reasons[col] = f"Missing {pct:.1%} y baja correlación (|corr| {abs_corr if not np.isnan(abs_corr) else 'NaN'} < {MIN_ABS_CORR_IMPORTANT})"
        continue
    # <5% -> no eliminar ahora (imputación futura), hacer nada

# 9. Aplicar eliminación
cols_drop = sorted(set(cols_drop))
print("\nResumen decisiones de eliminación:")
if cols_drop:
    for c in cols_drop:
        print(f" - {c}: {reasons.get(c,'(sin razón)')}")
else:
    print("No se eliminarán columnas bajo las reglas actuales.")

print(f"Total columnas originales: {len(original_columns)}")
print(f"Total a eliminar: {len(cols_drop)}")
print(f"Quedarán: {len(original_columns) - len(cols_drop)}")

# 10. Eliminar y guardar
if cols_drop:
    df.drop(columns=cols_drop, inplace=True, errors='ignore')
    df.to_csv(csv_file, index=False)
    print(f"Dataset guardado sin {len(cols_drop)} columnas en {csv_file}")
else:
    print("No se realizaron cambios en el CSV.")

# 11. (Opcional) Mostrar top correlaciones con TARGET_COL
if not corr_series.empty:
    print("\nTop 10 correlaciones (abs) con INDFMPIR:")
    top_corr = corr_series.reindex(corr_series.abs().sort_values(ascending=False).index)
    print(top_corr.head(10))

print("Proceso completado.")